# Data cleaning

In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\zhaoshuting\Documents\研究生\Module 3\Machine Learning in Finance\Fake Job\fake_job_postings.csv',engine ='python')

## 1.Brief unstanding of fake-jobposting dataset

In [3]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Unnamed: 18
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,NaN
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,NaN
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,located in Houston,is actively seeking an experienced Commission...,environmental,"and safety regulations.""",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI ??Environmental Systems Rese...,"EDUCATION:?Bachelor?? or Master?? in GIS, busi...",Our culture is anything but corporate??e have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,NaN
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,NaN


·job_id: Unique Job ID

·title: The title of the job ad entry.Most likely unique for each entry.

·location: Geographical location of the job ad：Country, State, City.

·department: Corporate department (e.g. sales).Most likely unique for each posting.

·salary_range: Indicative salary range (e.g. $50,000-$60,000).From an initial glance of the head, we see its blank; However, in subsequent analysis, we see that it is in format MIN-MAX

·company_profile: A brief company description.

·description: The details description of the job ad.

·requirements: Enlisted requirements for the job opening.

·benefits: Enlisted offered benefits by the employer.

·telecommuting: True for telecommuting positions.

·has_company_logo: True if company logo is present.

·has_questions: True if screening questions are present.

·employment_type: Full-type, Part-time, Contract, etc.

·required_experience: Executive, Entry level, Intern, etc.

·required_education: Doctorate, Master’s Degree, Bachelor, etc.

·industry: Automotive, IT, Health care, Real estate, etc.

·function: Consulting, Engineering, Research, Sales etc.

·fraudulent: target - Classification attribute.


## 2. Delete garbled lines

In [4]:
df.fraudulent.value_counts()

0                                                                                                                                                                                                                                                                                                                                                     16749
1                                                                                                                                                                                                                                                                                                                                                       895
 comprehensive                                                                                                                                                                                                                                                                                                  

We find there are strange values of column 'fraudulent' whose value should be 0 or 1. We decide to delete garbled lines.Because there are not many missing values of 'telecommuting', 'has_company_logo', 'has_questions', 'fraudulent', and the values of rows without misalignment are all 0 or 1, so directly delete the rows that are not 0 or 1.

In [5]:
df=df[(df.telecommuting=='1')|(df.telecommuting=='0')]
df=df[(df.has_company_logo=='1')|(df.has_company_logo=='0')]
df=df[(df.has_questions=='1')|(df.has_questions=='0')]
df=df[(df.fraudulent=='1')|(df.fraudulent=='0')]

## 3.Process features

In [6]:
df.nunique()

job_id                 17563
title                  11037
location                3065
department              1325
salary_range             867
company_profile         1703
description            14527
requirements           11713
benefits                6111
telecommuting              2
has_company_logo           2
has_questions              2
employment_type            5
required_experience        7
required_education        13
industry                 131
function                  37
fraudulent                 2
Unnamed: 18                0
dtype: int64

·'job_id' is unique for each sample and useless for detecting fake job.

·'title' is nearly unique and difficult to deal with.

·'location' is also difficult to handle and we want to find the generality of fake jobposting wherever it is.

·'department' is just like job title with many different values.

·'industry' and 'function' is also difficult to deal with and uselless for finding generality.

As a result, we decide to delete these six features.

In [7]:
drop_columns = ['job_id', 'title', 'location', 'department', 'industry', 'function','Unnamed: 18']

df = df.drop(drop_columns, axis=1)

In [8]:
df.isnull().sum()

salary_range           14722
company_profile         3279
description                1
requirements            2662
benefits                7012
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3393
required_experience     6908
required_education      7916
fraudulent                 0
dtype: int64

In [9]:
df.nunique()

salary_range             867
company_profile         1703
description            14527
requirements           11713
benefits                6111
telecommuting              2
has_company_logo           2
has_questions              2
employment_type            5
required_experience        7
required_education        13
fraudulent                 2
dtype: int64

We see that the 'employment_type', 'required_experience' and 'required_education' columns have a significant amout of missing values and only have 5, 7 and 13 unique object/categorical values, respectively. As these are manageable amounts, we replace
any missing values with a new object/category value of Unknown.

In [10]:
cat_columns = ['employment_type', 'required_experience', 'required_education']

for col in cat_columns:
    df[col].fillna("Unknown", inplace=True)

There are also missing values of text feature 'company_profile', 'description', 'requirements', 'benefits'. We decide to use ' ' to fill missing values. And if all these features are missing, we delete this sample.

In [11]:
text_columns = ['company_profile', 'description', 'requirements', 'benefits']

df = df.dropna(subset=text_columns, how='all')

for col in text_columns:
    df[col].fillna(' ', inplace=True)

salary interval is divided into minimum and maximum salary

In [12]:
new = df['salary_range'].str.split("-", n = 1, expand = True) 

df['salary_range_min']= new[0]
df['salary_range_max']= new[1]

df['salary_range_min'].fillna('-1', inplace=True)
df['salary_range_max'].fillna('-1', inplace=True)

def remove_string(x):
    if not x.isnumeric(): 
        val = '-1'
    else:
        val = x
    return val

df['salary_range_min'] = df['salary_range_min'].apply(lambda x: remove_string(x))
df['salary_range_max'] = df['salary_range_max'].apply(lambda x: remove_string(x))
df.drop('salary_range', axis=1, inplace = True)

In [13]:
df.head()

,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent,salary_range_min,salary_range_max
0,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,0,1,0,Other,Internship,Unknown,0,-1,-1
1,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Unknown,0,-1,-1
3,Our passion for improving quality of life thro...,THE COMPANY: ESRI ??Environmental Systems Rese...,"EDUCATION:?Bachelor?? or Master?? in GIS, busi...",Our culture is anything but corporate??e have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,0,-1,-1
4,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0,-1,-1
5,,Job OverviewApex is an environmental consultin...,,,0,0,0,Unknown,Unknown,Unknown,0,-1,-1


Move 'fraudulent' to the last column.

In [14]:
fraudulent=df.pop('fraudulent')
df.insert(12,'fraudulent',fraudulent)

Convert numeric variable format from str to numeric

In [15]:
df.fraudulent=pd.to_numeric(df.fraudulent)
df.telecommuting=pd.to_numeric(df.telecommuting)
df.has_company_logo=pd.to_numeric(df.has_company_logo)
df.has_questions=pd.to_numeric(df.has_questions)
df.salary_range_min=pd.to_numeric(df.salary_range_min)
df.salary_range_max=pd.to_numeric(df.salary_range_max)

In [16]:
outputpath=r'C:\Users\zhaoshuting\Documents\研究生\Module 3\Machine Learning in Finance\Fake Job\data cleaning\data for EDA.csv'
df.to_csv(outputpath,sep=',',index=False,header=True)